In [ ]:
import random
import matplotlib.pyplot as plt
import scipy.special as sps  
import numpy as np
import pandas as pd
from symfit import Parameter, Variable, parameters, variables, sin, cos, Fit
import statistics
import math

In [ ]:

def gettingstartpoints_EM(numberofcells = 50,weightscellcycledistributions=[100,0,0],
                       lambdaG1 = 1.15,kappaG1 = 9,lambdaS = 16.9,kappaS = 128,lambdaG2 = 5.59,kappaG2 = 19,lambdaM=0,kappaM=0,
                      outputpath='',treatment='DMSO'):
    import random
    import matplotlib.pyplot as plt
    import scipy.special as sps  
    import numpy as np

    data=[]
    phases=[]
    alllengths=[]
    lengthallG1=[]
    lengthallS=[]
    lengthallG2=[]
    lengthallM=[]

    for cell in list(range(1,int(numberofcells)+1)):
        starttime=0
        startphase = random.choices(["G1",'S','G2M'],weights=weightscellcycledistributions)
        lengthG1=np.random.gamma(shape=kappaG1,scale=1/lambdaG1)
        lengthS=np.random.gamma(shape=kappaS,scale=1/lambdaS)
        lengthG2=np.random.gamma(shape=kappaG2,scale=1/lambdaG2)
        lengthM=np.random.gamma(shape=kappaM,scale=1/lambdaM)
        maximallength=lengthG1+lengthS+lengthG2+lengthM
        if startphase[0] == 'G1':
            starttime=np.random.uniform(0,lengthG1,1)
        if startphase[0] == 'S':
            starttime=np.random.uniform(lengthG1,lengthG1+lengthS,1)
        if startphase[0] == 'G2M':
            starttime=np.random.uniform(lengthG1+lengthS,lengthG1+lengthS+lengthG2+lengthM,1)

        data +=[starttime]
        phases +=[startphase[0]]
        lengthallG1+=[lengthG1]
        lengthallS+=[lengthS]
        lengthallG2+=[lengthG2]
        lengthallM+=[lengthM]
        alllengths +=[maximallength]
    
    return [data,phases,alllengths,lengthallG1,lengthallS,lengthallG2,lengthallM]




In [ ]:
##For one cell the second cell cycle can vary

## --> Perminant stallment of population in G1!

def ReleaseIntoCellCycle_EM(data,phases,alllengthsinput,lengthallG1input,lengthallSinput,lengthallG2input,lengthallMinput,
                          lambdaG1_2nd = 8,kappaG1_2nd = 40,lambdaS_2nd = 16.9,kappaS_2nd = 128,lambdaG2_2nd = 5.59,kappaG2_2nd = 19,lambdaM_2nd=0,kappaM_2nd=0,
                          lambdaG1_3rd = 2.6,kappaG1_3rd = 30,lambdaS_3rd = 16.9,kappaS_3rd = 128,lambdaG2_3rd = 5.59,kappaG2_3rd = 19,lambdaM_3rd=0,kappaM_3rd=0,
                          progressionpercentage=100,progressionpercentage_2nd=100,
                          stallingcells_1st_G1=0,stallingcells_2nd_G1=0,stallingcells_3rd_G1=0,
                          stallingcells_1st_G2=0,stallingcells_2nd_G2=0,stallingcells_3rd_G2=0,
                          takingtimepoints=17,deltatime=3,
                          outputpath='',treatment='DMSO'):
    
    timecoursestart=[data]
    phasecycles = []
    phasecyclesRy=[]
    phasecyclesRx=[]
    countingdictionary={}
    realtimehours=0
    
    alllengths=alllengthsinput[:]
    alllengthsinput=alllengthsinput[:]
    lengthallG1=lengthallG1input[:]
    lengthallG1input=lengthallG1input[:]
    lengthallS=lengthallSinput[:]
    lengthallSinput=lengthallSinput[:]
    lengthallG2=lengthallG2input[:]
    lengthallG2input=lengthallG2input[:]
    lengthallM=lengthallMinput[:]
    lengthallMinput=lengthallMinput[:]
    
    modelledG1_2nd=[]
    modelledS_2nd=[]
    modelledG2_2nd=[]
    modelledM_2nd=[]
    modelledmaxlength_2nd=[]
    modelledG1_3rd=[]
    modelledS_3rd=[]
    modelledG2_3rd=[]
    modelledM_3rd=[]
    modelledmaxlength_3rd=[]

    G1startphasenumbers=0
    Sstartphasenumbers=0
    G2Mstartphasenumbers=0
    G2startphasenumbers=0
    Mstartphasenumbers=0
    cyclingorquiescentG1=[0]*len(timecoursestart[0])
    quiescentafterG1=[0]*takingtimepoints
    quiescentafterdictG1={0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0}
    errorcount=False
    
    for i in list(range(0,len(timecoursestart[0]))):
        startphase=phases[i]
        if startphase =="G1":
            G1startphasenumbers +=1
            cyclingorquiescentG1[i]=random.choices([True,False],weights=[100-stallingcells_1st_G1,stallingcells_1st_G1])[0]
            if cyclingorquiescentG1[i]==False:
                quiescentafterG1[0]+=1
                quiescentafterdictG1[0]+=1
        elif startphase =="S":
            Sstartphasenumbers +=1
            cyclingorquiescentG1[i]=True
        elif startphase =="G2M":
            G2Mstartphasenumbers+=1
            cyclingorquiescentG1[i]=True
            if timecoursestart[i] > (lengthallG1[i]+lengthallS[i]+lengthallG2[i]):
                Mstartphasenumbers +=1
            elif timecoursestart[i] < (lengthallG1[i]+lengthallS[i]+lengthallG2[i]):
                G2startphasenumbers +=1
            else:
                print('error0')
        else:
            print('error0')
                
    countingdictionary[0]=[G1startphasenumbers,Sstartphasenumbers,G2Mstartphasenumbers,G2startphasenumbers,Mstartphasenumbers,0]
    countskippingG1=0
    cellcyclenumber=[0]*len(timecoursestart[0])
    completedSphase=[0]*len(timecoursestart[0])
    completedSphasecountedonlyonce=[True]*len(timecoursestart[0])
    
    G2old=1000
    Mold=1000
    G1new=1000
    
    for timepoint in list(range(1,takingtimepoints)):
        timecoursetimepointnew=[]
        phasecyclesnew=[]
        countingdictionary[timepoint]=[0,0,0,0,0,0]
        realtimehours=realtimehours+deltatime
        countingdictionary[timepoint][5]=realtimehours
        progressionyes=0
        progressionno=0
     
        cellcount=[0]*takingtimepoints
        quiscentcellscount=[0]*takingtimepoints
        
        if min(alllengths) < deltatime:
            print('Error, time steps too big! Delta t is bigger then total cell cycle length!')
            errorcount=True
        
        for cell in range(0,len(timecoursestart[timepoint-1])):
            cellcount[timepoint]+=1
            
            if alllengths[cell] < deltatime:
                print('Error, time steps too big! Delta t is bigger then total cell cycle length!')
                errorcount=True
            if deltatime > (G2old +Mold + G1new) and cellcyclenumber[cell] >0:
                print('Error, time steps too big! Delta t is bigger then the interphase between S phases!')
                errorcount=True
            
            if cyclingorquiescentG1[cell] == True:
                progressionstep = random.choices([True,False],weights=[progressionpercentage,100-progressionpercentage])

                progression=timecoursestart[timepoint-1][cell]+deltatime
                if progressionstep[0] == True:
                    progression=timecoursestart[timepoint-1][cell]+deltatime
                    progressionyes+=1
                    if progression < alllengths[cell]: ##It is the first cell cycle
                        phasecyclesRx+=[timepoint]

                        if progression < alllengths[cell]:
                            timecoursetimepointnew+=[progression]
                            correctedprogression=progression
                        else:
                            print('error1')

                        if correctedprogression < lengthallG1[cell]:
                            phasecyclesnew +=['G1']
                            phasecyclesRy +=['G1']
                            countingdictionary[timepoint][0]+=1
                            if completedSphasecountedonlyonce[cell] == False:
                                completedSphase[cell]+=1
                                completedSphasecountedonlyonce[cell]=True
                            completedSphasecountedonlyonce[cell] =True

                        elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]):
                            phasecyclesnew +=['S']
                            phasecyclesRy +=['S']
                            countingdictionary[timepoint][1]+=1
                            completedSphasecountedonlyonce[cell] =False
                        
                        elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]):
                            phasecyclesnew +=['G2M']
                            phasecyclesRy +=['G2M']
                            countingdictionary[timepoint][2]+=1
                            countingdictionary[timepoint][3]+=1 ##G2 phase
                        
                            if completedSphasecountedonlyonce[cell] == False:
                                completedSphase[cell]+=1
                                completedSphasecountedonlyonce[cell]=True
                        
                        elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]+lengthallM[cell]):
                            phasecyclesnew +=['G2M']
                            phasecyclesRy +=['G2M']
                            countingdictionary[timepoint][2]+=1
                            countingdictionary[timepoint][4]+=1 ##M phase

                            if completedSphasecountedonlyonce[cell] == False:
                                completedSphase[cell]+=1
                                completedSphasecountedonlyonce[cell]=True

                        else:
                            phasecyclesnew +=['Error']
                            phasecyclesRy +=['Error']        
                            print('error2')

                    elif progression >= alllengths[cell]: ##It is the not first cell cycle:
                        if cellcyclenumber[cell] ==0: ##It is second cell cycle
                            cellcyclenumber[cell]+=1
                            phasecyclesRx+=[timepoint]

                            timecoursetimepointnew+=[progression-alllengths[cell]]
                            correctedprogression=progression-alllengths[cell]
                            G2old=lengthallG2[cell]
                            Mold=lengthallM[cell]
                            
                            lengthG1_2nd=np.random.gamma(shape=kappaG1_2nd,scale=1/lambdaG1_2nd)
                            lengthS_2nd=np.random.gamma(shape=kappaS_2nd,scale=1/lambdaS_2nd)
                            lengthG2_2nd=np.random.gamma(shape=kappaG2_2nd,scale=1/lambdaG2_2nd)
                            lengthM_2nd=np.random.gamma(shape=kappaM_2nd,scale=1/lambdaM_2nd)
                            maximallength_2nd=lengthG1_2nd+lengthS_2nd+lengthG2_2nd+lengthM_2nd
                            
                            G1new=lengthG1_2nd
                            
                            lengthallG1[cell]=lengthG1_2nd
                            lengthallS[cell]=lengthS_2nd
                            lengthallG2[cell]=lengthG2_2nd
                            lengthallM[cell]=lengthM_2nd
                            alllengths[cell]=maximallength_2nd
                            modelledG1_2nd.append(lengthG1_2nd)
                            modelledS_2nd.append(lengthS_2nd)
                            modelledG2_2nd.append(lengthG2_2nd)
                            modelledM_2nd.append(lengthM_2nd)
                            modelledmaxlength_2nd.append(maximallength_2nd)
                            
                            cyclingorquiescentG1[cell]=random.choices([True,False],weights=[100-stallingcells_2nd_G1,stallingcells_2nd_G1])[0]
                            
                            if cyclingorquiescentG1[cell]==False:
                                quiescentafterG1[1]+=1
                                quiescentafterdictG1[1]+=1
                            
                            if correctedprogression < lengthallG1[cell]:
                                phasecyclesnew +=['G1']
                                phasecyclesRy +=['G1']
                                countingdictionary[timepoint][0]+=1
                                if completedSphasecountedonlyonce[cell] == False:
                                    completedSphase[cell]+=1
                                    completedSphasecountedonlyonce[cell]=True

                            elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]):
                                phasecyclesnew +=['S']
                                phasecyclesRy +=['S']
                                countskippingG1+=1
                                countingdictionary[timepoint][1]+=1

                            elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]):
                                phasecyclesnew +=['G2M']
                                phasecyclesRy +=['G2M']
                                print('skipSandG1')
                                countingdictionary[timepoint][2]+=1
                                countingdictionary[timepoint][3]+=1
                            
                            elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]+lengthallM[cell]):
                                phasecyclesnew +=['G2M']
                                phasecyclesRy +=['G2M']
                                print('skipSandG1')
                                countingdictionary[timepoint][2]+=1
                                countingdictionary[timepoint][4]+=1

                            else:
                                phasecyclesnew +=['Error']
                                phasecyclesRy +=['Error']        
                                print('error3')
                        elif cellcyclenumber[cell] >0: ##It is 3rd or more cell cycle
                            cellcyclenumber[cell]+=1
                            phasecyclesRx+=[timepoint]
                            timecoursetimepointnew+=[progression-alllengths[cell]]
                            correctedprogression=progression-alllengths[cell]

                            #Corrected progression is progression in the new cell cycle. Assuming that no cell cycle is 3 hours.N
                            
                            G2old=lengthallG2[cell]
                            Mold=lengthallM[cell]
                            
                            lengthG1_3rd=np.random.gamma(shape=kappaG1_3rd,scale=1/lambdaG1_3rd)
                            lengthS_3rd=np.random.gamma(shape=kappaS_3rd,scale=1/lambdaS_3rd)
                            lengthG2_3rd=np.random.gamma(shape=kappaG2_3rd,scale=1/lambdaG2_3rd)
                            lengthM_3rd=np.random.gamma(shape=kappaM_3rd,scale=1/lambdaM_3rd)
                            maximallength_3rd=lengthG1_3rd+lengthS_3rd+lengthG2_3rd+lengthM_3rd
                            
                            G1new=lengthG1_3rd
                            
                            lengthallG1[cell]=lengthG1_3rd
                            lengthallS[cell]=lengthS_3rd
                            lengthallG2[cell]=lengthG2_3rd
                            lengthallM[cell]=lengthM_3rd
                            alllengths[cell]=maximallength_3rd
                            
                            modelledG1_3rd.append(lengthG1_3rd)
                            modelledS_3rd.append(lengthS_3rd)
                            modelledG2_3rd.append(lengthG2_3rd)
                            modelledM_3rd.append(lengthM_3rd)
                            modelledmaxlength_3rd.append(maximallength_3rd)
                            
                            cyclingorquiescentG1[cell]=random.choices([True,False],weights=[100-stallingcells_3rd_G1,stallingcells_3rd_G1])[0]
                            if cyclingorquiescentG1[cell]==False:
                                quiescentafterG1[cellcyclenumber[cell]]+=1
                                
                                if cellcyclenumber[cell] in quiescentafterdictG1:
                                    quiescentafterdictG1[cellcyclenumber[cell]]+=1
                                else:
                                    quiescentafterdictG1[int(cellcyclenumber[cell])]=1

                            if correctedprogression < lengthallG1[cell]:
                                phasecyclesnew +=['G1']
                                phasecyclesRy +=['G1']
                                countingdictionary[timepoint][0]+=1 
                                
                                if completedSphasecountedonlyonce[cell] == False:
                                    completedSphase[cell]+=1
                                    completedSphasecountedonlyonce[cell]=True

                            elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]):
                                phasecyclesnew +=['S']
                                phasecyclesRy +=['S']
                                countskippingG1+=1
                                countingdictionary[timepoint][1]+=1

                            elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]):
                                phasecyclesnew +=['G2M']
                                phasecyclesRy +=['G2M']
                                print('skipSandG1')
                                countingdictionary[timepoint][2]+=1
                                countingdictionary[timepoint][3]+=1
                            
                            elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]+lengthallM[cell]):
                                phasecyclesnew +=['G2M']
                                phasecyclesRy +=['G2M']
                                print('skipSandG1')
                                countingdictionary[timepoint][2]+=1
                                countingdictionary[timepoint][4]+=1

                            else:
                                phasecyclesnew +=['Error']
                                phasecyclesRy +=['Error']        
                                print('error4')
                            
                    else:
                        print('error5')
                else:
                    progression=timecoursestart[timepoint-1][cell]
                    timecoursetimepointnew+=[progression]
                    progressionno+=1
                    correctedprogression=progression
                    if correctedprogression < lengthallG1[cell]:
                        phasecyclesnew +=['G1']
                        phasecyclesRy +=['G1']
                        countingdictionary[timepoint][0]+=1

                    elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]):
                        phasecyclesnew +=['S']
                        phasecyclesRy +=['S']

                        countingdictionary[timepoint][1]+=1

                    elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]):
                        phasecyclesnew +=['G2M']
                        phasecyclesRy +=['G2M']
                        countingdictionary[timepoint][2]+=1
                        countingdictionary[timepoint][3]+=1
                    elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]+lengthallM[cell]):
                        phasecyclesnew +=['G2M']
                        phasecyclesRy +=['G2M']
                        countingdictionary[timepoint][2]+=1
                        countingdictionary[timepoint][4]+=1

                    else:
                        phasecyclesnew +=['Error']
                        phasecyclesRy +=['Error']        
                        print('error5')
                        
            else:
                quiscentcellscount[timepoint]+=1
                progression=timecoursestart[timepoint-1][cell]
                timecoursetimepointnew+=[progression]
                
                correctedprogression=progression
                if correctedprogression < lengthallG1[cell]:
                    phasecyclesnew +=['G1']
                    phasecyclesRy +=['G1']
                    countingdictionary[timepoint][0]+=1
                elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]):
                    phasecyclesnew +=['S']
                    phasecyclesRy +=['S']
                    countingdictionary[timepoint][1]+=1
                elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]):
                    phasecyclesnew +=['G2M']
                    phasecyclesRy +=['G2M']
                    countingdictionary[timepoint][2]+=1
                    countingdictionary[timepoint][3]+=1
                elif correctedprogression < (lengthallG1[cell]+lengthallS[cell]+lengthallG2[cell]+lengthallM[cell]):
                    phasecyclesnew +=['G2M']
                    phasecyclesRy +=['G2M']
                    countingdictionary[timepoint][2]+=1
                    countingdictionary[timepoint][4]+=1
                else:
                    phasecyclesnew +=['Error']
                    phasecyclesRy +=['Error']        
                    print('error6')

        timecoursestart+=[timecoursetimepointnew]
        phasecycles+=[phasecyclesnew]
    import pandas as pd
    
    df= pd.DataFrame.from_dict(countingdictionary, orient='index',columns=['G1','S','G2M','G2','M','time'])

    return df,errorcount

        
        


In [ ]:
## 

print('starting')

numberofcells=5000
takingtimepoints=49
deltatimeinterval=1
weightscellcycledistributions=[100,0,0]
outputpath='C:/Users/Gebruiker/OneDrive - Universiteit Utrecht/CSDB/Major Internship/Modelling cell cycle/TempFigures/'
inputpath='C:/Users/Gebruiker/OneDrive - Universiteit Utrecht/CSDB/Major Internship/Modelling cell cycle/'



treatment='DMSO'
lambdaG1_1st=1.15
kappaG1_1st=9
lambdaS_1st=16.9
kappaS_1st=128
lambdaG2_1st=5.6
kappaG2_1st=19
lambdaM_1st=35.5
kappaM_1st=16
####
lambdaG1_2nd=1.15
kappaG1_2nd=9
lambdaS_2nd=16.9
kappaS_2nd=128
lambdaG2_2nd=5.6
kappaG2_2nd=19
lambdaM_2nd=35.5
kappaM_2nd=16
####
lambdaG1_3rd=1.15
kappaG1_3rd=9
lambdaS_3rd=16.9
kappaS_3rd=128
lambdaG2_3rd=5.6
kappaG2_3rd=19
lambdaM_3rd=35.5
kappaM_3rd=16
####
progression_1st=100
progression_2nd=100
stallingcells_1st=20
stallingcells_2nd=30
stallingcells_3rd=30
variablespositionlistlambda=[float(lambdaG1_1st),float(lambdaS_1st),float(lambdaG2_1st),float(lambdaG1_2nd),float(lambdaS_2nd),float(lambdaG2_2nd),
                             float(lambdaG1_3rd),float(lambdaS_3rd),float(lambdaG2_3rd)]
variablespositionlistkappa=[kappaG1_1st,kappaS_1st,kappaG2_1st,kappaG1_2nd,kappaS_2nd,kappaG2_2nd,
                           kappaG1_3rd,kappaS_3rd,kappaG2_3rd]
variablesdimensions=9

oldRsquare=-10
newtotalRsquare=-100
selectedcount=0
declinedcount=0
accepteddecrease=0
factorA=0.01
FormSampling1=True
FormSampling2=False
FormSampling3=False
countsmallsteps=0
countbigsteps=0

list_lambdaG1_1st=[]
list_kappaG1_1st=[]
list_lambdaS_1st=[]
list_kappaS_1st=[]
list_lambdaG2_1st=[]
list_kappaG2_1st=[]
list_lambdaG1_2nd=[]
list_kappaG1_2nd=[]
list_lambdaS_2nd=[]
list_kappaS_2nd=[]
list_lambdaG2_2nd=[]
list_kappaG2_2nd=[]
list_lambdaG1_3rd=[]
list_kappaG1_3rd=[]
list_lambdaS_3rd=[]
list_kappaS_3rd=[]
list_lambdaG2_3rd=[]
list_kappaG2_3rd=[]

list_Rsquares=[]
list_Rsquareacceptance=[] #2 means increase. 1 means decrease but accepted. -1 means not accepted.
list_Rsquaresselected=[]
list_itterationnumber=[]
list_formofsampling=[] #1 average step. 2 means small steps. 3 means big steps. 4 'crazy' steps.
list_clustering=[]

list_scatterR_kappaandlambdas_values=[] 
list_scatterR_kappaandlambdas=[] 
list_scatterR_Rsquares=[]
list_scatterR_formofsampling=[]
list_scatterR_itterationnumber=[]
list_scatterR_Rsquareacceptance=[]
list_scatterR_cellcyclephase=[]
list_scatterR_kappa_values=[]
list_scatterR_lambda_values=[]
list_scatterR_clustering=[]

list_scatterR_cellcyclelength=[]
list_scatterR_cellcyclelength_1st=[]
list_scatterR_cellcyclelength_2nd=[]
list_scatterR_cellcyclelength_3rd=[]
list_scatterR_cycle=[]
list_scatterR_phaselength=[]


upperbordersearchkappa=300
upperbordersearchlambda=30
if treatment == 'DMSO':
    minimallength_3rd=15
    maximallength_3rd=28
elif treatment == 'dTAG-V1':
    minimallength_3rd=24
    maximallength_3rd=37
else:
    print('error treatment not found')
minimumlength_1stG1=1
maximumlength_1stG1=18


clustering=1

i=0
while i <2000:
    i+=1
    list_itterationnumber+=[i]
    list_scatterR_itterationnumber+=[i]*variablesdimensions
    list_clustering+=[clustering]
    list_scatterR_clustering+=[clustering]*variablesdimensions
    accepted=False
    

    if i in list(range(0,5000,100)):
        print('itteration: ',i)

    variablespositionlistlambda_temp=variablespositionlistlambda[:]
    variablespositionlistkappa_temp=variablespositionlistkappa[:]
    
    if treatment == 'DMSO':
        variabletochange=random.choices([0,1,2,3,4,5,6,7,8])[0] ##Which cell cycle hpases can be changed.
    elif treatment =='dTAG-V1':
        variabletochange=random.choices([1,3,4,6,7])[0]
    else:
        print('no treatment found')
    formsampling=-10
    steppossible=False
    n=0
    while steppossible ==False:
        steppossible =True
        n+=1

        if FormSampling1==True:
            bothkappaandlambda=True
            formsampling=1
            if newtotalRsquare <0.50:
                factorA=0.03
                
                if variabletochange !=-1:
                    if variablespositionlistkappa[variabletochange] <40 or variablespositionlistkappa[variabletochange] > upperbordersearchkappa-40:
                        valuesaddkappa=random.choices([1,2,3,5,7,10,25])[0]
                    elif variablespositionlistkappa[variabletochange] <80 or variablespositionlistkappa[variabletochange] > upperbordersearchkappa-80:
                        valuesaddkappa=random.choices([3,5,10,20,50])[0]
                    else:
                        valuesaddkappa=random.choices([10,20,30,40,50])[0]

                    if variablespositionlistlambda[variabletochange] <5 or variablespositionlistlambda[variabletochange] > upperbordersearchlambda-5:
                        valuesaddlambda=random.choices([0.10,0.25,0.50,0.70,1.00,3.00])[0]
                    elif variablespositionlistlambda[variabletochange] <20 or variablespositionlistlambda[variabletochange] > upperbordersearchlambda-20:
                        valuesaddlambda=random.choices([0.10,0.25,0.50,1.00,2.00,3.00,4.00,5.00])[0]
                    else:    
                        valuesaddlambda=random.choices([2.00,4.00,5.00,7.00,8.00])[0]
                else:
                    valuesaddkappa=random.choices([0.1,0.2,0.3,0.5,1,2])[0]
                    valuesaddlambda=random.choices([0.01,0.05,0.1,0.2,0.5,1])[0]
            else:
                factorA=0.03
                if variabletochange !=-1:
                    if variablespositionlistkappa[variabletochange] <40 or variablespositionlistkappa[variabletochange] > upperbordersearchkappa-40:
                        valuesaddkappa=random.choices([1,2,3,5,20])[0]
                    elif variablespositionlistkappa[variabletochange] <80 or variablespositionlistkappa[variabletochange] > upperbordersearchkappa-80:
                        valuesaddkappa=random.choices([3,5,10,20,35])[0]
                    else:
                        valuesaddkappa=random.choices([10,20,30,40])[0]

                    if variablespositionlistlambda[variabletochange] <5 or variablespositionlistlambda[variabletochange] > upperbordersearchlambda-5:
                        valuesaddlambda=random.choices([0.10,0.25,0.50,0.70,1.50])[0]
                    elif variablespositionlistlambda[variabletochange] <20 or variablespositionlistlambda[variabletochange] > upperbordersearchlambda-20:
                        valuesaddlambda=random.choices([0.10,0.25,0.50,1.00,2.00,3.00,5.00])[0]
                    else:    
                        valuesaddlambda=random.choices([1.00,2.00,4.00,5.00,8.00])[0]
                else:
                    valuesaddkappa=random.choices([0.1,0.2,0.3,0.5,1,2])[0]
                    valuesaddlambda=random.choices([0.01,0.05,0.1,0.2,0.5,1])[0]
        elif FormSampling2==True:
            declinedcount=0
            if variabletochange !=-1:
                valuesaddlambda=random.choices([0,0,0.10,0.25,0.50,1.00,1.00,3.00])[0]
                valuesaddkappa=random.choices([0,0,1,2.5,5,10,10])[0]
            else:
                valuesaddkappa=random.choices([0.05,0.1,0.2,0.3])[0]
                valuesaddlambda=random.choices([0.01,0.05,0.1])[0]
            bothkappaandlambda=True
            factorA=0.01
            formsampling=2
            countsmallsteps+=1

        elif FormSampling3==True:
            if variabletochange !=-1:
                valuesaddlambda=random.choices([5.00,6.00,7.00,8.00])[0]
                valuesaddkappa=random.choices([60,70,80])[0]
            else:
                valuesaddkappa=random.choices([0.1,0.2,0.3,0.5,1,2])[0]
                valuesaddlambda=random.choices([0.01,0.05,0.1,0.2,0.5,1])[0]
            bothkappaandlambda=True
            factorA=0.13
            formsampling=3
            countbigsteps+=1
        else:
            print('error form sampling',FormSampling1,FormSampling2,FormSampling3)
            valuesaddlambda=random.choices([1,3,5,6,7,8,10,12,15,20,25])[0]
            valuesaddkappa=random.choices([20,60,70,80,90,100,110,120,130,140,150])[0]
            bothkappaandlambda=True
            formsampling=4

        if bothkappaandlambda == True:
            increasekappa =variablespositionlistkappa[variabletochange] + valuesaddkappa
            decreasekappa=variablespositionlistkappa[variabletochange] - valuesaddkappa
            increaselambda=variablespositionlistlambda[variabletochange] + valuesaddlambda
            decreaselambda=variablespositionlistlambda[variabletochange] - valuesaddlambda
            
            if variabletochange in [1,2,3,4,5,6,7,8]:
                maxlength=24
                minlength=1
            else:
                maxlength=maximumlength_1stG1
                minlength=minimumlength_1stG1
                    
            
            if variablespositionlistkappa[variabletochange] <100 and variablespositionlistlambda[variabletochange] <10 and variabletochange in [1,2,3,4,5,6,7,8]:

                #increase
                upperborderlambdaincrease = increasekappa/minlength
                lowerborderlambdaincrease = increasekappa/maxlength
                upperborderkappaincrease = increaselambda*maxlength
                lowerborderkappaincrease = increaselambda*minlength
                #increase
                upperborderlambdadecrease = decreasekappa/minlength
                lowerborderlambdadecrease = decreasekappa/maxlength
                upperborderkappadecrease = decreaselambda*maxlength
                lowerborderkappadecrease = decreaselambda*minlength

                
                if increasekappa>100 or increaselambda>10:
                    if increasekappa>100 and increaselambda<=10:
                        increasedecrease=False
                        if decreasekappa > 1:
                            decreaseincrease=True
                            if decreaselambda>1:
                                decreasedecrease=True
                            else:
                                decreasedecrease=False
                            if upperborderkappaincrease > increasekappa and lowerborderkappaincrease < increasekappa:
                                increaseincrease=True
                            else:
                                increaseincrease=False
                        else:
                            decreaseincrease=False
                            decreasedecrease=False
                            if upperborderkappaincrease > increasekappa and lowerborderkappaincrease < increasekappa:
                                increaseincrease=True
                            else:
                                increaseincrease=False
                                                        
                    elif increasekappa<=100 and increaselambda>10:
                        decreaseincrease=False
                        if decreaselambda>1:
                            increasedecrease=True
                            if decreasekappa>1:
                                decreasedecrease=True
                            else:
                                decreasedecrease=False
                            if upperborderlambdaincrease > increaselambda and lowerborderlambdaincrease < increaselambda:
                                increaseincrease=True
                            else:
                                increaseincrease=False
                        else:
                            decreasedecrease=False
                            increasedecrease=False
                            if upperborderlambdaincrease > increaselambda and lowerborderlambdaincrease < increaselambda:
                                increaseincrease=True
                            else:
                                increaseincrease=False
                            
                            
                    elif increasekappa>100 and increaselambda>10:
                        ##verzinnen.
                        #increase increase
                        if increasekappa > lowerborderkappa and increasekappa < upperborderkappa and increaselambda > lowerborderlambda and increaselambda < upperborderlambda:
                            increaseincrease=True
                        else:
                            increaseincrease=False
                            
                        if decreasekappa > lowerborderkappa and decreasekappa < upperborderkappa and increaselambda > lowerborderlambda and increaselambda < upperborderlambda:
                            decreaseincrease=True
                        else:
                            decreaseincrease=False
                            
                        if increasekappa > lowerborderkappa and increasekappa < upperborderkappa and decreaselambda > lowerborderlambda and decreaselambda < upperborderlambda:
                            increasedecrease=True
                        else:
                            increasedecrease=False
                        
                        if decreasekappa >1 and decreaselambda >1:
                            decreasedecrease=True
                        else:
                            decreasedecrease=False
                    else:
                        print('ERRROR')
                
                            
                    if increaseincrease ==True:
                        if decreaseincrease==True:
                            if increasedecrease==True:
                                if decreasedecrease==True:
                                    substractoraddkappa=random.choice([-1,1])
                                    substractoraddlambda=random.choice([-1,1])
                                else:
                                    substractoraddkappa=random.choice([-1,1])
                                    if substractoraddkappa==1:
                                        substractoraddlambda=random.choice([-1,1])
                                    else:
                                        substractoraddlambda=1
                                        
                            else:
                                if decreasedecrease ==True:
                                    substractoraddkappa=random.choice([-1,1])
                                    if substractoraddkappa==1:
                                        substractoraddlambda=1
                                    else:
                                        substractoraddlambda=random.choice([-1,1])
                                else:
                                    substractoraddkappa=random.choice([-1,1])
                                    substractoraddlambda=1
                        else:
                            if increasedecrease==True:
                                if decreasedecrease==True:
                                    substractoraddkappa=random.choice([-1,1])
                                    if substractoraddkappa==1:
                                        substractoraddlambda=random.choice([-1,1])
                                    else:
                                        substractoraddlambda=-1
                                else:
                                    substractoraddkappa=1
                                    substractoraddlambda=random.choice([-1,1])
                            else:
                                if decreasedecrease==True:
                                    substractoraddkappa=random.choice([-1,1])
                                    substractoraddkappa=substractoraddlambda
                                else:
                                    substractoraddkappa=1
                                    substractoraddlambda=1
                    else:
                        if decreaseincrease ==True:
                            if increasedecrease==True:
                                if decreasedecrease==True:
                                    substractoraddkappa=random.choice([-1,1])
                                    if substractoraddkappa==1:
                                        substractoraddlambda=-1
                                    else:
                                        substractoraddlambda=random.choice([-1,1])
                                else:
                                    substractoraddkappa=random.choice([-1,1])
                                    if substractoraddkappa==1:
                                        substractoraddlambda=-1
                                    else:
                                        substractoraddlambda=1
                            else:
                                if decreasedecrease ==True:
                                    substractoraddkappa=-1
                                    substractoraddlambda=random.choice([-1,1])
                                else:
                                    substractoraddkappa=-1
                                    substractoraddlambda=1
                        else:
                            if increasedecrease==True:
                                if decreasedecrease==True:
                                    substractoraddkappa=random.choice([-1,1])
                                    substractoraddlambda=-1
                                else:
                                    substractoraddkappa=1
                                    substractoraddlambda=-1
                            else:
                                if decreasedecrease==True:
                                    substractoraddkappa=-1
                                    substractoraddlambda=-1
                                else:
                                    steppossible=False
                                    print("ERRROR No step possible :( within 100 and 10")
                    
                else: ## steps in bigger area!
                    if decreasekappa >1:
                        substractoraddkappa=random.choice([-1,1])
                    else:
                        substractoraddkappa=1 
                    if decreaselambda>1:
                        substractoraddlambda=random.choice([-1,1])
                    else:
                        substractoraddlambda=1
                
                if steppossible==True:
                    variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoraddkappa)
                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoraddlambda),2)

            else:
                #increase increase
                upperborderlambda = increasekappa/minlength
                lowerborderlambda = increasekappa/maxlength
                upperborderkappa = increaselambda*maxlength
                lowerborderkappa = increaselambda*minlength
                                
                if upperborderkappa>upperbordersearchkappa:
                    upperborderkappa=upperbordersearchkappa
                if upperborderlambda>upperbordersearchlambda:
                    upperborderlambda=upperbordersearchlambda     

                if increasekappa > lowerborderkappa and increasekappa < upperborderkappa and increaselambda > lowerborderlambda and increaselambda < upperborderlambda:
                    increaseincrease=True
                else:
                    increaseincrease=False

                #decrease decrease
                upperborderlambda = decreasekappa/minlength
                lowerborderlambda = decreasekappa/maxlength
                upperborderkappa = decreaselambda*maxlength
                lowerborderkappa = decreaselambda*minlength

                if decreasekappa > lowerborderkappa and decreasekappa < upperborderkappa and decreaselambda > lowerborderlambda and decreaselambda < upperborderlambda:
                    decreasedecrease=True
                else:
                    decreasedecrease=False

                #decrease kappa increase lambda. decrease increase
                upperborderlambda = decreasekappa/minlength
                lowerborderlambda = decreasekappa/maxlength
                upperborderkappa = increaselambda*maxlength
                lowerborderkappa = increaselambda*minlength

                if upperborderlambda>upperbordersearchlambda:
                    upperborderlambda=upperbordersearchlambda 

                if decreasekappa > lowerborderkappa and decreasekappa < upperborderkappa and increaselambda > lowerborderlambda and increaselambda < upperborderlambda:
                    decreaseincrease=True
                else:
                    decreaseincrease=False

                #increase kappa decrease lambda. increase decrease
                upperborderlambda = increasekappa/minlength
                lowerborderlambda = increasekappa/maxlength
                upperborderkappa = decreaselambda*maxlength
                lowerborderkappa = decreaselambda*minlength

                if upperborderkappa>upperbordersearchkappa:
                    upperborderkappa=upperbordersearchkappa
                
                if increasekappa > lowerborderkappa and increasekappa < upperborderkappa and decreaselambda > lowerborderlambda and decreaselambda < upperborderlambda:
                    increasedecrease=True
                else:
                    increasedecrease=False

                if increaseincrease ==True:
                    if decreasedecrease==True:
                        if increasedecrease==True:
                            if decreaseincrease==True:
                                substractoradd=random.choice([-1,1])
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                substractoradd=random.choice([-1,1])
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                            else:
                                substractoradd=random.choice([-1,1]) #for kappa
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                if substractoradd==1:
                                    substractoradd=random.choice([-1,1]) #for lambda
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                else:
                                    substractoradd=-1 #for lambda
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                        else:
                            if decreaseincrease==True:
                                substractoradd=random.choice([-1,1]) #for kappa
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                if substractoradd==-1:
                                    substractoradd=random.choice([-1,1]) #for lambda
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                else:
                                    substractoradd=1 #for lambda
                                    variablespositionlistlambda_temp[variabletochange]=variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd)
                            else:
                                substractoradd=random.choice([-1,1])
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)                                  
                    else:
                        if increasedecrease==True:
                            if decreaseincrease==True: #for kappa
                                substractoradd=random.choice([-1,1])
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                if substractoradd==1: #for lambda
                                    substractoradd=random.choice([-1,1])
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                else:
                                    substractoradd=1 #for lambda
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                            else:
                                substractoradd=1 #for kappa
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                substractoradd=random.choice([-1,1]) #for lambda
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                        else:
                            if decreaseincrease==True:
                                substractoradd=random.choice([-1,1]) #for kappa
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                substractoradd=1 #for lambda
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                            else:
                                substractoradd=1 #for kappa and lambda
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                else:
                    if decreasedecrease ==True:
                        if increasedecrease ==True:
                            if decreaseincrease==True:
                                substractoradd=random.choice([-1,1]) #for kappa
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                if substractoradd ==-1: #for lambda
                                    substractoradd=random.choice([-1,1])
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                else:
                                    substractoradd=-1
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                            else:
                                substractoradd=random.choice([-1,1])
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                substractoradd=-1
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                        else:
                            if decreaseincrease==True:
                                substractoradd=-1
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                substractoradd=random.choice([-1,1])
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                            else:
                                substractoradd=-1
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                    else:
                        if increasedecrease ==True:
                            if decreaseincrease ==True:
                                substractoradd=random.choice([-1,1])
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                if substractoradd==1:
                                    substractoradd=-1
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                                else:
                                    substractoradd=1
                                    variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                            else:
                                substractoradd=1
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                substractoradd=-1
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                        else:
                            if decreaseincrease ==True:
                                substractoradd=-1
                                variablespositionlistkappa_temp[variabletochange]=variablespositionlistkappa[variabletochange]+(valuesaddkappa*substractoradd)
                                substractoradd=1
                                variablespositionlistlambda_temp[variabletochange]=round(variablespositionlistlambda[variabletochange]+(valuesaddlambda*substractoradd),2)
                            else:
                                print("ERRROR No step possible :( within larger area")
                                steppossible=False
        if n >500:
            steppossible=True
            print('kan niet goed zijn')

     
    list_formofsampling+=[formsampling]
    list_scatterR_formofsampling+=[formsampling]*variablesdimensions
    list_lambdaG1_1st+=[variablespositionlistlambda_temp[0]]
    list_kappaG1_1st+=[variablespositionlistkappa_temp[0]]
    list_lambdaS_1st+=[variablespositionlistlambda_temp[1]]
    list_kappaS_1st+=[variablespositionlistkappa_temp[1]]
    list_lambdaG2_1st+=[variablespositionlistlambda_temp[2]]
    list_kappaG2_1st+=[variablespositionlistkappa_temp[2]]
    list_lambdaG1_2nd+=[variablespositionlistlambda_temp[3]]
    list_kappaG1_2nd+=[variablespositionlistkappa_temp[3]]
    list_lambdaS_2nd+=[variablespositionlistlambda_temp[4]]
    list_kappaS_2nd+=[variablespositionlistkappa_temp[4]]
    list_lambdaG2_2nd+=[variablespositionlistlambda_temp[5]]
    list_kappaG2_2nd+=[variablespositionlistkappa_temp[5]]
    list_lambdaG1_3rd+=[variablespositionlistlambda_temp[6]]
    list_kappaG1_3rd+=[variablespositionlistkappa_temp[6]]
    list_lambdaS_3rd+=[variablespositionlistlambda_temp[7]]
    list_kappaS_3rd+=[variablespositionlistkappa_temp[7]]
    list_lambdaG2_3rd+=[variablespositionlistlambda_temp[8]]
    list_kappaG2_3rd+=[variablespositionlistkappa_temp[8]]
    
    list_scatterR_kappaandlambdas_values+=[variablespositionlistlambda_temp[0],variablespositionlistkappa_temp[0],
                                         variablespositionlistlambda_temp[1],variablespositionlistkappa_temp[1],
                                         variablespositionlistlambda_temp[2],variablespositionlistkappa_temp[2],
                                          variablespositionlistlambda_temp[3],variablespositionlistkappa_temp[3],
                                          variablespositionlistlambda_temp[4],variablespositionlistkappa_temp[4],
                                          variablespositionlistlambda_temp[5],variablespositionlistkappa_temp[5],
                                          variablespositionlistlambda_temp[6],variablespositionlistkappa_temp[6],
                                          variablespositionlistlambda_temp[7],variablespositionlistkappa_temp[7],
                                         variablespositionlistlambda_temp[8],variablespositionlistkappa_temp[8]]
    list_scatterR_kappaandlambdas+=['lambdaG1_1st','kappaG1_1st','lambdaS_1st','kappaS_1st','lambdaG2_1st','kappaG2_1st',
                                  'lambdaG1_2nd','kappaG1_2nd','lambdaS_2nd','kappaS_2nd','lambdaG2_2nd','kappaG2_2nd',
                                  'lambdaG1_3rd','kappaG1_3rd','lambdaS_3rd','kappaS_3rd','lambdaG2_3rd','kappaG2_3rd']
    
    list_scatterR_cellcyclephase += ['G1_1st','S_1st','G2_1st','G1_2nd','S_2nd','G2_2nd','G1_3rd','S_3rd','G2_3rd']
    
    list_scatterR_kappa_values += [variablespositionlistkappa_temp[0],variablespositionlistkappa_temp[1],
                                 variablespositionlistkappa_temp[2],variablespositionlistkappa_temp[3],
                                 variablespositionlistkappa_temp[4],variablespositionlistkappa_temp[5],
                                 variablespositionlistkappa_temp[6],variablespositionlistkappa_temp[7],
                                  variablespositionlistkappa_temp[8]]
    
    list_scatterR_lambda_values += [variablespositionlistlambda_temp[0],variablespositionlistlambda_temp[1],
                                 variablespositionlistlambda_temp[2],variablespositionlistlambda_temp[3],
                                 variablespositionlistlambda_temp[4],variablespositionlistlambda_temp[5],
                                 variablespositionlistlambda_temp[6],variablespositionlistlambda_temp[7],
                                  variablespositionlistlambda_temp[8]]
    
    outputPalbo=gettingstartpoints_EM(numberofcells = numberofcells,
                                      lambdaG1 = variablespositionlistlambda_temp[0],kappaG1 = variablespositionlistkappa_temp[0],lambdaS = variablespositionlistlambda_temp[1],kappaS = variablespositionlistkappa_temp[1],
                                      lambdaG2 = variablespositionlistlambda_temp[2],kappaG2 = variablespositionlistkappa_temp[2],lambdaM = lambdaM_1st,kappaM = kappaM_1st,
                                      outputpath=outputpath,treatment=treatment)
    #print(min(outputPalbo[2]))
    if min(outputPalbo[2])>deltatimeinterval and min(outputPalbo[3])+min(outputPalbo[5])+min(outputPalbo[6]) > deltatimeinterval:
        dfPalbo,errorsfound=ReleaseIntoCellCycle_EM(outputPalbo[0],outputPalbo[1],outputPalbo[2],outputPalbo[3],outputPalbo[4],outputPalbo[5],outputPalbo[6],
                                        lambdaG1_2nd = variablespositionlistlambda_temp[3],kappaG1_2nd = variablespositionlistkappa_temp[3],lambdaS_2nd = variablespositionlistlambda_temp[4],kappaS_2nd = variablespositionlistkappa_temp[4],lambdaG2_2nd = variablespositionlistlambda_temp[5],kappaG2_2nd = variablespositionlistkappa_temp[5],lambdaM_2nd = lambdaM_2nd,kappaM_2nd = kappaM_2nd,
                                        lambdaG1_3rd = variablespositionlistlambda_temp[6],kappaG1_3rd = variablespositionlistkappa_temp[6],lambdaS_3rd = variablespositionlistlambda_temp[7],kappaS_3rd = variablespositionlistkappa_temp[7],lambdaG2_3rd = variablespositionlistlambda_temp[8],kappaG2_3rd = variablespositionlistkappa_temp[8],lambdaM_3rd = lambdaM_3rd,kappaM_3rd = kappaM_3rd,
                                        progressionpercentage=progression_1st,progressionpercentage_2nd=progression_2nd,
                                        stallingcells_1st_G1=stallingcells_1st,stallingcells_2nd_G1=stallingcells_2nd,stallingcells_3rd_G1=stallingcells_3rd,
                                        takingtimepoints=takingtimepoints,deltatime=deltatimeinterval,
                                        outputpath=outputpath,treatment=treatment)

        concatdf = pd.concat([dfPalbo], axis=1)
        concatdf=concatdf.set_index('time')
        concatdf=concatdf/numberofcells*100
        
        ## checking if the 3rd cell cycle is between given ranges.
        lengthG1_1st=(variablespositionlistkappa_temp[0]/variablespositionlistlambda_temp[0])*(1-(1/(3*variablespositionlistkappa_temp[0]+0.2)))
        lengthS_1st=(variablespositionlistkappa_temp[1]/variablespositionlistlambda_temp[1])*(1-(1/(3*variablespositionlistkappa_temp[1]+0.2)))
        lengthG2_1st=(variablespositionlistkappa_temp[2]/variablespositionlistlambda_temp[2])*(1-(1/(3*variablespositionlistkappa_temp[2]+0.2)))
        lengthM_1st=(kappaM_1st/lambdaM_1st)*(1-(1/(3*kappaM_1st+0.2)))
        length1stcycle=lengthG1_1st+lengthS_1st+lengthG2_1st+lengthM_1st
        
        lengthG1_2nd=(variablespositionlistkappa_temp[3]/variablespositionlistlambda_temp[3])*(1-(1/(3*variablespositionlistkappa_temp[3]+0.2)))
        lengthS_2nd=(variablespositionlistkappa_temp[4]/variablespositionlistlambda_temp[4])*(1-(1/(3*variablespositionlistkappa_temp[4]+0.2)))
        lengthG2_2nd=(variablespositionlistkappa_temp[5]/variablespositionlistlambda_temp[5])*(1-(1/(3*variablespositionlistkappa_temp[5]+0.2)))
        lengthM_2nd=(kappaM_2nd/lambdaM_2nd)*(1-(1/(3*kappaM_2nd+0.2)))
        length2ndcycle=lengthG1_2nd+lengthS_2nd+lengthG2_2nd+lengthM_2nd
        
        lengthG1_3rd=(variablespositionlistkappa_temp[6]/variablespositionlistlambda_temp[6])*(1-(1/(3*variablespositionlistkappa_temp[6]+0.2)))
        lengthS_3rd=(variablespositionlistkappa_temp[7]/variablespositionlistlambda_temp[7])*(1-(1/(3*variablespositionlistkappa_temp[7]+0.2)))
        lengthG2_3rd=(variablespositionlistkappa_temp[8]/variablespositionlistlambda_temp[8])*(1-(1/(3*variablespositionlistkappa_temp[8]+0.2)))
        lengthM_3rd=(kappaM_3rd/lambdaM_3rd)*(1-(1/(3*kappaM_3rd+0.2)))
        length3rdcycle=lengthG1_3rd+lengthS_3rd+lengthG2_3rd+lengthM_3rd
        
        list_scatterR_cellcyclelength+=[length1stcycle,length1stcycle,length1stcycle,length2ndcycle,length2ndcycle,length2ndcycle,length3rdcycle,length3rdcycle,length3rdcycle]
        list_scatterR_cellcyclelength_1st+=[length1stcycle]*variablesdimensions
        list_scatterR_cellcyclelength_2nd+=[length2ndcycle]*variablesdimensions
        list_scatterR_cellcyclelength_3rd+=[length3rdcycle]*variablesdimensions
        list_scatterR_cycle+=[1,1,1,2,2,2,3,3,3]
        list_scatterR_phaselength+=[lengthG1_1st,lengthS_1st,lengthG2_1st,
                                   lengthG1_2nd,lengthG1_2nd,lengthG2_2nd,
                                   lengthG1_3rd,lengthS_3rd,lengthG2_3rd]
        
        
        if treatment=='DMSO':
            concatdf.columns = ['G1 DMSO', 'S DMSO', 'G2M DMSO','G2','M']
            concatdf['Time'] = concatdf.index

            realdata = pd.read_csv(str(inputpath+'ExperimentalData.txt'),sep='\t')
            SSresG1DMSO = 0
            SStotG1DMSO = 0
            SSresSDMSO = 0
            SStotSDMSO = 0
            SSresG2MDMSO = 0
            SStotG2MDMSO = 0
            SSresG1V1 = 0
            SStotG1V1 = 0
            SSresSV1 = 0
            SStotSV1 = 0
            SSresG2MV1 = 0
            SStotG2MV1 = 0
            SSresTotalDMSO=0
            SSresTotalV1=0
            SStotTotalDMSO=0
            SStotTotalV1=0

            modeldata=concatdf.iloc[::3,:]

            G1modelDMSO=modeldata['G1 DMSO'].tolist()
            G1expDMSO=realdata['G1 DMSO exp'].tolist()
            #G1modelV1=modeldata['G1 dTAG-V1'].tolist()
            #G1expV1=realdata['G1 dTAG-V1 exp'].tolist()
            SmodelDMSO=modeldata['S DMSO'].tolist()
            SexpDMSO=realdata['S DMSO exp'].tolist()
            #SmodelV1=modeldata['S dTAG-V1'].tolist()
            #SexpV1=realdata['S dTAG-V1 exp'].tolist()
            G2MmodelDMSO=modeldata['G2M DMSO'].tolist()
            G2MexpDMSO=realdata['G2M DMSO exp'].tolist()
            #G2MmodelV1=modeldata['G2M dTAG-V1'].tolist()
            #G2MexpV1=realdata['G2M dTAG-V1 exp'].tolist()
        elif treatment=='dTAG-V1':
            concatdf.columns = ['G1 dTAG-V1', 'S dTAG-V1', 'G2M dTAG-V1','G2','M']
            concatdf['Time'] = concatdf.index

            realdata = pd.read_csv(str(inputpath+'ExperimentalData.txt'),sep='\t')
            SSresG1DMSO = 0
            SStotG1DMSO = 0
            SSresSDMSO = 0
            SStotSDMSO = 0
            SSresG2MDMSO = 0
            SStotG2MDMSO = 0
            SSresG1V1 = 0
            SStotG1V1 = 0
            SSresSV1 = 0
            SStotSV1 = 0
            SSresG2MV1 = 0
            SStotG2MV1 = 0
            SSresTotalDMSO=0
            SSresTotalV1=0
            SStotTotalDMSO=0
            SStotTotalV1=0
            
            modeldata=concatdf.iloc[::3,:]
            
            G1modelDMSO=modeldata['G1 dTAG-V1'].tolist()
            G1expDMSO=realdata['G1 dTAG-V1 exp'].tolist()
            #G1modelV1=modeldata['G1 dTAG-V1'].tolist()
            #G1expV1=realdata['G1 dTAG-V1 exp'].tolist()
            SmodelDMSO=modeldata['S dTAG-V1'].tolist()
            SexpDMSO=realdata['S dTAG-V1 exp'].tolist()
            #SmodelV1=modeldata['S dTAG-V1'].tolist()
            #SexpV1=realdata['S dTAG-V1 exp'].tolist()
            G2MmodelDMSO=modeldata['G2M dTAG-V1'].tolist()
            G2MexpDMSO=realdata['G2M dTAG-V1 exp'].tolist()
            #G2MmodelV1=modeldata['G2M dTAG-V1'].tolist()
            #G2MexpV1=realdata['G2M dTAG-V1 exp'].tolist()


        loop=0
        for loop in range(0,len(G1expDMSO)):
            SSresG1DMSO+=(G1expDMSO[loop]-G1modelDMSO[loop])**2
            SStotG1DMSO+=(G1expDMSO[loop]-statistics.mean(G1expDMSO))**2
            SSresSDMSO+=(SexpDMSO[loop]-SmodelDMSO[loop])**2
            SStotSDMSO+=(SexpDMSO[loop]-statistics.mean(SexpDMSO))**2
            SSresG2MDMSO+=(G2MexpDMSO[loop]-G2MmodelDMSO[loop])**2
            SStotG2MDMSO+=(G2MexpDMSO[loop]-statistics.mean(G2MexpDMSO))**2


            SSresTotalDMSO=SSresG1DMSO+SSresSDMSO+SSresG2MDMSO
            SStotTotalDMSO=SStotG1DMSO+SStotSDMSO+SStotG2MDMSO

        newtotalRsquare=1-(SSresTotalDMSO/SStotTotalDMSO)
        list_Rsquares+=[newtotalRsquare]
        list_scatterR_Rsquares+=[newtotalRsquare]*variablesdimensions
        differenceRsquares=oldRsquare-newtotalRsquare
        
        if length3rdcycle<minimallength_3rd or length3rdcycle>maximallength_3rd:
            errorsfound = True
            
        
        if oldRsquare < newtotalRsquare and errorsfound == False:
            oldRsquare=newtotalRsquare
            variablespositionlistlambda=variablespositionlistlambda_temp
            variablespositionlistkappa=variablespositionlistkappa_temp
            print('R square total: ',1-(SSresTotalDMSO/SStotTotalDMSO),'\t ACCEPTED',SSresTotalDMSO,SStotTotalDMSO,'\t',formsampling,'\t',declinedcount,countsmallsteps,countbigsteps)
            declinedcount=0
            list_Rsquareacceptance+=[2]
            list_scatterR_Rsquareacceptance+=[2]*variablesdimensions
            accepted=True
                        
        elif oldRsquare >= newtotalRsquare and errorsfound == False:
            if random.random() < math.e**(-differenceRsquares/factorA):
                oldRsquare=newtotalRsquare
                variablespositionlistlambda=variablespositionlistlambda_temp
                variablespositionlistkappa=variablespositionlistkappa_temp
                print('R square total: ',1-(SSresTotalDMSO/SStotTotalDMSO),'\t ACCEPTED DECREASE\t',SSresTotalDMSO,SStotTotalDMSO,'\t',formsampling,'\t',declinedcount,countsmallsteps,countbigsteps)
                declinedcount=0
                list_Rsquareacceptance+=[1]
                list_scatterR_Rsquareacceptance+=[1]*variablesdimensions
                accepteddecrease+=1
                accepted=True
            else:
                list_Rsquareacceptance+=[-1]
                list_scatterR_Rsquareacceptance+=[-1]*variablesdimensions
                declinedcount+=1
                accepted=False
            #print('R square total: ',1-(SSresTotalDMSO/SStotTotalDMSO),'\t DECLINED',declinedcount)
        elif errorsfound == True:
            declinedcount+=1
            list_Rsquareacceptance+=[-2]
            list_scatterR_Rsquareacceptance+=[-2]*variablesdimensions
            newRsquare=-100
            accepted=False
        
        if newtotalRsquare >= 0.85:
            selectedcount+=1
      
    else:
        print('jammer')
        declinedcount+=1
        list_Rsquareacceptance+=[-2]
        list_scatterR_Rsquareacceptance+=[-2]*variablesdimensions
        newRsquare=-100
        accepted=False
        
    if newtotalRsquare>0.70 and FormSampling1==True:
        FormSampling2=True
        FormSampling1=False
        FromSampling3=False
        countsmallsteps=0
        
    elif FormSampling2==True:
        if countsmallsteps>299 and newtotalRsquare <0.80:
            FormSampling2=False
            FormSampling1=False
            FormSampling3=True
            countsmallsteps=0
            contbigsteps=0
            clustering+=1
        elif countsmallsteps> 399:
            FormSampling2=False
            FormSampling1=False
            FormSampling3=True
            countsmallsteps=0
            contbigsteps=0
            clustering+=1
        
    if countbigsteps>49:
        FormSampling3=False
        FormSampling2=False
        FormSampling1=True
        countbigsteps=0
        
    elif newtotalRsquare<0.4 and accepted==True:
        FormSampling3=False
        FormSampling2=False
        FormSampling1=True
        countbigsteps=0

print('lambda',variablespositionlistlambda)
print('kappa',variablespositionlistkappa)
print('counts of Rsquare above 0.85: ',selectedcount)
print('counts of accepted decrease: ',accepteddecrease)




In [ ]:
df = pd.DataFrame(list(zip(list_itterationnumber,list_formofsampling,list_Rsquares,list_Rsquareacceptance,list_clustering,
                           list_lambdaG1_1st, list_kappaG1_1st,list_lambdaS_1st,list_kappaS_1st,
                           list_lambdaG2_1st,list_kappaG2_1st,list_lambdaG1_2nd,list_kappaG1_2nd,
                           list_lambdaS_2nd,list_kappaS_2nd,list_lambdaG2_2nd,list_kappaG2_2nd,
                           list_lambdaG1_3rd,list_kappaG1_3rd,list_lambdaS_3rd,list_kappaS_3rd,
                           list_lambdaG2_3rd,list_kappaG2_3rd)),
                  columns =['Itteration', 'FormofSampling','Rsquares','Rsquaresacceptance','Cluster','lambdaG1_1st','kappaG1_1st',
                        'lambdaS_1st','kappaS_1st','lambdaG2_1st','kappaG2_1st','lambdaG1_2nd','kappaG1_2nd',
                        'lambdaS_2nd','kappaS_2nd','lambdaG2_2nd','kappaG2_2nd','lambdaG1_3rd','kappaG1_3rd',
                        'lambdaS_3rd','kappaS_3rd','lambdaG2_3rd','kappaG2_3rd'])

df
outputpath='C:/Users/Gebruiker/OneDrive - Universiteit Utrecht/CSDB/Major Internship/Modelling cell cycle/'
df.to_csv(str(outputpath+'tableX.txt'), header=True, index=None, sep='\t')



dfR = pd.DataFrame(list(zip(list_scatterR_itterationnumber,list_scatterR_formofsampling,list_scatterR_Rsquares,
                            list_scatterR_Rsquareacceptance,list_scatterR_clustering,
                            list_scatterR_cycle,list_scatterR_cellcyclephase,list_scatterR_lambda_values,list_scatterR_kappa_values,
                            list_scatterR_phaselength,list_scatterR_cellcyclelength,list_scatterR_cellcyclelength_1st,list_scatterR_cellcyclelength_2nd,
                           list_scatterR_cellcyclelength_3rd)),
                  columns =['Itteration', 'FormofSampling','Rsquares','Rsquaresacceptance','Cluster','Cycle','CellCyclePhase',
                            'LambdaValue','KappaValue','PhaseLength','CellCycleLength','CellCycleLength_1st','CellCycleLength_2nd',
                           'CellCycleLength_3rd'])

dfR
outputpath='C:/Users/Gebruiker/OneDrive - Universiteit Utrecht/CSDB/Major Internship/Modelling cell cycle/'
dfR.to_csv(str(outputpath+'tableX_forR.txt'), header=True, index=None, sep='\t')
